In [54]:
import hd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import importlib

from tensorflow.python.client import timeline

In [55]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.graph_options.optimizer_options.global_jit_level = tf.OptimizerOptions.ON_1
sess = tf.Session(config=config)
cpu_session = tf.Session(config=tf.ConfigProto(device_count={'GPU': 0}))
run_metadata = tf.RunMetadata()

In [56]:
vectors = hd.vectors.space_graph_altered_permutations([5, 5, 3, 2, 1, 1], bounds=(0.0, 1.0))

In [57]:
sess.run(vectors)

array([[-5., -5.,  2.,  2.,  1.,  0.],
       [-5., -5.,  3.,  1.,  0.,  1.],
       [-5., -5.,  3.,  1.,  1.,  0.],
       ...,
       [ 5.,  5., -3., -2.,  1., -1.],
       [ 5.,  5., -1., -1., -1., -1.],
       [ 5.,  5.,  0., -2., -1., -1.]])

In [58]:
root_valences = hd.tenney.hd_root_valence(vectors)
downhill = tf.boolean_mask(vectors, root_valences < 0.0)

In [59]:
downhill_vecs = sess.run(downhill)

In [60]:
downhill_vecs.shape

(714, 6)

In [91]:
hds = hd.tenney.hd_aggregate_graph(downhill[:, None, :])
reasonable = tf.boolean_mask(downhill, hds < 8.0)

In [92]:
out = sess.run(reasonable)

In [93]:
valence_array = sess.run(hd.tenney.hd_root_valence(reasonable))

In [94]:
def vector_to_ratio(vector):
    primes = hd.PRIMES[:vector.shape[0]]
    num = np.where(vector > 0, vector, np.zeros_like(primes))
    den = np.where(vector < 0, vector, np.zeros_like(primes))
    return (
        np.product(np.power(primes, num)), 
        np.product(primes ** np.abs(den))
    )

all_possible_pitches = { }

for row, valence in zip(out, valence_array):
    all_possible_pitches[vector_to_ratio(row)] = valence

print(len(all_possible_pitches))
print(sorted(all_possible_pitches, key=lambda r: (r[0] / r[1])))

18
[(16.0, 15.0), (14.0, 13.0), (12.0, 11.0), (10.0, 9.0), (8.0, 7.0), (6.0, 5.0), (16.0, 13.0), (14.0, 11.0), (4.0, 3.0), (18.0, 13.0), (10.0, 7.0), (16.0, 11.0), (14.0, 9.0), (8.0, 5.0), (18.0, 11.0), (12.0, 7.0), (16.0, 9.0), (20.0, 11.0)]


In [95]:
idx_orders = np.argsort(list(all_possible_pitches.values()))

In [96]:
for row in out[idx_orders]:
    print(vector_to_ratio(row))

(16.0, 15.0)
(16.0, 13.0)
(16.0, 11.0)
(16.0, 9.0)
(8.0, 7.0)
(8.0, 5.0)
(12.0, 11.0)
(4.0, 3.0)
(12.0, 7.0)
(20.0, 11.0)
(14.0, 13.0)
(10.0, 9.0)
(6.0, 5.0)
(14.0, 11.0)
(18.0, 13.0)
(10.0, 7.0)
(14.0, 9.0)
(18.0, 11.0)
